In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline


from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
random_state = 0

In [4]:
df_train = pd.read_csv('dataset_diabetes/diabetic_data_train.csv')

In [5]:
df_train = df_train[[col for col in df_train.columns if col not in ['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty', 'examide', 'citoglipton']]]

## Create Labels

In [6]:
[(key, val/len(df_train.readmitted)) for (key, val) in Counter(df_train.readmitted).most_common()]

[('NO', 0.5393799439886012),
 ('>30', 0.3491868520611212),
 ('<30', 0.1114332039502776)]

In [7]:
def binarize_readmitted(x):
    if x in ['<30']: # readmitted in less than 30 days is positive class
        return 1 
    else:
        return 0 

In [8]:
y_train = df_train.apply(lambda x: binarize_readmitted(x['readmitted']), axis=1).values

In [9]:
[(key, val/len(df_train.readmitted)) for (key, val) in Counter(y_train).most_common()]

[(0, 0.8885667960497224), (1, 0.1114332039502776)]

## Process Features

In [10]:
def map_age(x):
    if x == '[0-10)':
        return 5
    elif x == '[10-20)':
        return 15
    elif x == '[20-30)':
        return 25
    elif x == '[30-40)':
        return 35
    elif x == '[40-50)':
        return 45
    elif x == '[50-60)':
        return 55
    elif x == '[60-70)':
        return 65
    elif x == '[70-80)':
        return 75
    elif x == '[80-90)':
        return 85
    else:
        return 95

admission_dict = {1: 'Emergency', 2: 'Urgent', 3: 'Elective', 4: 'Newborn', 7: 'Trauma Center'}
def map_admission(x):
    if x in [5,6,8]:
        return 'Not Available'
    else:
        return admission_dict[x]
    
def discharged_home(x):
    if x == 1:
        return 1 # Discharged home
    else:
        return 0 # Other 

def map_source(x):
    if x in [1,2,3]:
        return 'Referral' 
    elif x == 7:
        return 'Emergency Room'
    else:
        return 'Other'

def map_diag(x):
    if x[0].isdigit() == False:
        return 'other'
    
    float_x = float(x)
    if (float_x >= 390 and float_x <= 459) or float_x == 785: # 390–459, 785
        return 'circulatory'
    elif (float_x >= 460 and float_x <= 519) or float_x == 786: # 460–519, 786
        return 'respiratory'
    elif (float_x >= 520 and float_x <= 579) or float_x == 787: # 520–579, 787
        return 'digestive'
    elif float_x >= 250 and float_x < 251: # 250.xx
        return 'diabetes'
    elif float_x >= 800 and float_x <= 999: # 800–999
        return 'injury'
    elif float_x >= 710 and float_x <= 739: # 710–739
        return 'musculoskeletal'
    elif (float_x >= 580 and float_x <= 629) or float_x == 788: # 580–629, 788
        return 'genitourinary'
    elif float_x >= 140 and float_x <= 239: # 140–239
        return 'neoplasms'
    else:
        return 'other'

def binarize_yn(x):
    if x in ['Yes', 'Ch']: 
        return 1 
    else:
        return 0 

In [11]:
df_train['age_processed'] = df_train.apply(lambda x: map_age(x['age']), axis=1)
df_train['admission_type_processed'] = df_train.apply(lambda x: map_admission(x['admission_type_id']), axis=1)
df_train['discharged_processed'] = df_train.apply(lambda x: discharged_home(x['discharge_disposition_id']), axis=1)
df_train['source_processed'] = df_train.apply(lambda x: map_source(x['admission_source_id']), axis=1)

In [12]:
df_train['diag1_processed'] = df_train.apply(lambda x: map_diag(x['diag_1']), axis=1)
df_train['diag2_processed'] = df_train.apply(lambda x: map_diag(x['diag_2']), axis=1)
df_train['diag3_processed'] = df_train.apply(lambda x: map_diag(x['diag_3']), axis=1)
df_train['change_processed'] = df_train.apply(lambda x: binarize_yn(x['change']), axis=1)
df_train['diabetesMed_processed'] = df_train.apply(lambda x: binarize_yn(x['diabetesMed']), axis=1)

## One Hot Encodings

In [13]:
ohe = OneHotEncoder(categories='auto')

In [14]:
df_train_sub = df_train[['admission_type_processed', 'source_processed',
    'diag1_processed', 'diag2_processed', 'diag3_processed', 'race', 'gender', 
    'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 
    'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 
    'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 
    'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 
    'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone']]

I would get rid of chlorpropamide, acetohexamide, tolbutamide, acarbose, miglitol, troglitazone, tolazamide, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, metformin-pioglitazone

In [15]:
train_ohe = ohe.fit_transform(df_train_sub).toarray()

In [16]:
feature_labels_ = ohe.categories_

In [17]:
feature_labels = []
for col, values in zip(df_train_sub.columns, feature_labels_):
    for val in values:
        feature_labels.append(col+'_'+val)

In [18]:
df_train_ohe = pd.DataFrame(train_ohe, columns = feature_labels)

In [19]:
cols = ['age_processed', 'discharged_processed', 'change_processed', 'diabetesMed_processed', 
        'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 
        'number_outpatient','number_emergency', 'number_inpatient', 'number_diagnoses']

df_train_num = df_train[cols]

In [20]:
print(df_train_ohe.shape, df_train_num.shape)

(81412, 122) (81412, 12)


In [21]:
X_train = pd.concat([df_train_ohe, df_train_num], axis=1)

## Cross Validation Pipeline

In [22]:
pipe = Pipeline([
        ('sampling', SMOTE(random_state=random_state)),
        ('model', RandomForestClassifier(random_state=random_state))
    ])

pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'sampling', 'model', 'sampling__k_neighbors', 'sampling__n_jobs', 'sampling__random_state', 'sampling__sampling_strategy', 'model__bootstrap', 'model__ccp_alpha', 'model__class_weight', 'model__criterion', 'model__max_depth', 'model__max_features', 'model__max_leaf_nodes', 'model__max_samples', 'model__min_impurity_decrease', 'model__min_impurity_split', 'model__min_samples_leaf', 'model__min_samples_split', 'model__min_weight_fraction_leaf', 'model__n_estimators', 'model__n_jobs', 'model__oob_score', 'model__random_state', 'model__verbose', 'model__warm_start'])

In [23]:
param_grid = {
    'model__n_estimators': [100, 500],
    'model__max_depth': [4, 6, 8, 10, 12, 14],
    'model__random_state': [random_state]
}

scoring = ['accuracy', 'roc_auc', 'recall']

In [24]:
clf = GridSearchCV(pipe, param_grid, cv=4, verbose=5, scoring=scoring, refit='roc_auc', return_train_score=True)

In [ ]:
best_model = clf.fit(X_train, y_train)